<a href="https://colab.research.google.com/github/djyamunaq/RobotArm6dof/blob/main/6dof_Robot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
from math import cos, sin, pi

In [ ]:
# DH parameters
a = [0.0, 0.0, 0.0, 0.0825, -0.0825, 0, 0.088]
d = [0.333, 0.0, 0.316, 0, 0.384, 0, 0]
alpha = [0, -pi/2, pi/2, pi/2, -pi/2, pi/2, pi/2, 0]

In [ ]:
def calculate_position(qIn):
  Ts = []

  T = np.identity(4)

  # ...----o----... => ...(Link i-1)(Joint i)(Link i)...
  # For each link i
  for i in range(0, 7):
    l0 = [cos(qIn[i]), -sin(qIn[i]), 0, a[i]]
    l1 = [sin(qIn[i])*cos(alpha[i]), cos(qIn[i])*cos(alpha[i]), -sin(alpha[i]), -d[i]*sin(alpha[i])]
    l2 = [sin(qIn[i])*sin(alpha[i]), cos(qIn[i])*sin(alpha[i]), cos(alpha[i]), d[i]*cos(alpha[i])]
    l3 = [0, 0, 0, 1]
    Tcurr = np.matrix([l0, l1, l2, l3])
    Tcurr[abs(Tcurr) < 1e-10] = 0

    Ts.append(Tcurr.copy())

    T = np.dot(T, Tcurr)

  # for i in range(0, 7):
  #   print('=======================================================================')
  #   print('T(', i, ', ', i+1, ')')
  #   print(Ts[i])

  # print('=======================================================================')
  # print('T(', 0, ', ', 7, ')')
  T[abs(T) < 1e-10] = 0
  # print(T)
  return T

In [ ]:
# Jacobian calculation
  # Jpi = Zi-1 x (pe - pi-1)
  # Joi = Zi-1
  # Zi = 0R1(q1) * ... * i-1Ri(qi-1)*z0 (z0 = [0 0 1]' --> 3rd col)
  # pe = 0Te * [0 0 0 1] (4th col of Transformation Matrix)
  # pi-1 = 0Ti-1 * [0 0 0 1] (4th col of Transformation Matrix from frame 0 to i-1)
  # J = [Jp Jo]' =
  # dp/dt = sum from i=0 to n(Jpi x dq/dt)
  # w = sum from i=0 to n(Joi * dq/dt)

J = np.zeros([6, 7])

pe = np.array([T[i, -1] for i in range(0, 3)])
p0 = np.array([0, 0, 0])
z0 = np.array([0, 0, 1])

# Calculate each J(i+1)
for i in range(0, 7):
  # print('=== Joint ', i, '===')

  # Calculate Zi
  X = np.identity(4)
  for j in range(0, i+1):
    # Transformation Matrix jT(j+1)
    X = X*Ts[j]

  z = np.array([X[j, -2].copy() for j in range(0, 3)])
  p = np.array([X[j, -1].copy() for j in range(0, 3)])

  Jpi = np.cross(z, (pe - p))
 
  # print(Jpi, end='\n\n')

  Joi = z.copy()

  # print('====== J', i, '======')
  J[:, i] = np.concatenate([Jpi, Joi]) 
  # print(J[:, i-1])

J


array([[ 0.    ,  0.7   ,  0.    , -0.384 ,  0.    , -0.    ,  0.    ],
       [ 0.088 ,  0.    ,  0.088 ,  0.    ,  0.088 ,  0.    , -0.    ],
       [ 0.    , -0.088 ,  0.    ,  0.0055,  0.    ,  0.088 ,  0.    ],
       [ 0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ],
       [ 0.    ,  1.    ,  0.    , -1.    ,  0.    , -1.    ,  0.    ],
       [ 1.    ,  0.    ,  1.    ,  0.    ,  1.    ,  0.    , -1.    ]])

In [ ]:
# End-effector position
pe = np.array([0.088, 0, 1.033])

# Joint 1
T01 = Ts[0]
z1 = np.array([T01[j, -2].copy() for j in range(0, 3)])
p1 = np.array([T01[j, -1].copy() for j in range(0, 3)])

Jp1 = np.cross(z1, (pe - p1))

print('=== Joint 1 ===')
print(Jp1)

# Joint 2
T02 = T01*Ts[1]
z2 = np.array([T02[j, -2].copy() for j in range(0, 3)])
p2 = np.array([T02[j, -1].copy() for j in range(0, 3)])

Jp2 = np.cross(z2, (pe - p2))

print('=== Joint 2 ===')
print(Jp2)

# Joint 3
T03 = T02*Ts[2]
z3 = np.array([T03[j, -2].copy() for j in range(0, 3)])
p3 = np.array([T03[j, -1].copy() for j in range(0, 3)])

Jp3 = np.cross(z3, (pe - p3))

print('=== Joint 3 ===')
print(Jp3)

# Joint 4
T04 = T03*Ts[3]
z4 = np.array([T04[j, -2].copy() for j in range(0, 3)])
p4 = np.array([T04[j, -1].copy() for j in range(0, 3)])

Jp4 = np.cross(z4, (pe - p4))

print('=== Joint 4 ===')
print(Jp4)

# Joint 5
T05 = T04*Ts[4]
z5 = np.array([T05[j, -2].copy() for j in range(0, 3)])
p5 = np.array([T05[j, -1].copy() for j in range(0, 3)])

Jp5 = np.cross(z5, (pe - p5))

print('=== Joint 5 ===')
print(Jp5)

# Joint 6
T06 = T05*Ts[5]
z6 = np.array([T06[j, -2].copy() for j in range(0, 3)])
p6 = np.array([T06[j, -1].copy() for j in range(0, 3)])

Jp6 = np.cross(z6, (pe - p6))

print('=== Joint 6 ===')
print(Jp6)

# Joint 5
T07 = T06*Ts[6]
z7 = np.array([T07[j, -2].copy() for j in range(0, 3)])
p7 = np.array([T07[j, -1].copy() for j in range(0, 3)])

Jp7 = np.cross(z7, (pe - p7))

print('=== Joint 7 ===')
print(Jp7)






=== Joint 1 ===
[0.    0.088 0.   ]
=== Joint 2 ===
[ 0.7    0.    -0.088]
=== Joint 3 ===
[0.    0.088 0.   ]
=== Joint 4 ===
[-0.384   0.      0.0055]
=== Joint 5 ===
[0.    0.088 0.   ]
=== Joint 6 ===
[-0.     0.     0.088]
=== Joint 7 ===
[ 0. -0.  0.]


In [ ]:
np.cross(np.array([0, 0, 1]), np.array([0.088, 0, 0.7]))

In [ ]:
# Initial position
qi = np.array([0, 0, 0, 0, 0, 0, 0])
Ti = calculate_position(qi)
pi = np.array([Ti[j, -1] for j in range(0, 3)])
pi_m = np.array([-0.8177, 0, 1.0317])

# Final position
qf = np.array([1.21691,  0.20459,  2.66903,  1.59144, -3.04646,  1.77385, 0.732767])
Tf = calculate_position(qf)
pf = np.array([Tf[j, -1] for j in range(0, 3)])
pf_m = np.array([-0.9028, 0.1143, 1.1156])

print('Initial position:')
print(pi)
print('\nFinal position:')
print(pf)
print('\Measured delta:')
print(pf_m - pi_m)



Initial position:
[0.088 0.    1.033]

Final position:
[0.29976437 0.29974973 0.49972058]
\Measured delta:
[-0.0851  0.1143  0.0839]
